In [7]:
 # if plotting does not work, change it into %matplotlib inline
%matplotlib inline

In [8]:
import numpy as np
import matplotlib.pyplot as plt
from utilities import display_array
import deepdish as dd

# Load the data

In [9]:
fps = 1.5

Change this folder to the one where you have the dataset

In [10]:
data_path = r"/Users/chenchen/Documents/MSNE/Semester4/2.Large_Scale_Modeling/msne-lsmlsda-2019/your-code/project3_zhenchen/exercise/week11_imaging_analysis/imaging_analysis/2p_nuclear/imaging.h5"

In [11]:
ar_imaging = dd.io.load(data_path)

In [12]:
display_array(ar_imaging);

interactive(children=(IntSlider(value=279, description='t', max=559), IntSlider(value=1, description='z', max=…

The array you obtained has dimensions [time, plane, y, x]

## Average in time to get a cleaner anatomy

In [13]:
mean=np.mean(ar_imaging,axis=0)
anatomy=ar_imaging-mean

In [14]:
display_array(anatomy, vmax=150)

interactive(children=(IntSlider(value=279, description='t', max=559), IntSlider(value=1, description='z', max=…

<function utilities.display_array.<locals>.browse(t:(0, 559), z:(0, 2))>

## Find the correlations with neighbouring pixels

In [ ]:
from scipy.stats import zscore

First, normalize time series of each pixel so that they are 0 centered and have standard deviation 0 (the function imported above does that for you). This makes calculating correlations between pixel time traces trivial (look up the definition of Person's correlation coefficient)

Correlate each pixel with it's neighbours and add correlations. Hint: you need only one for loop that goes through 4 iterations, use slicing.

In [ ]:
display_array(correlations)

## Find local maxima

In [ ]:
from skimage.feature import peak_local_max

In [ ]:
i_plane = 0

The function imporeted above finds local maxima. Look at it's documentation. It will not work well across planes, so do the analysis plane by plane

You should obtain something like this:
![](correlation_local_maxima.png)

In [ ]:
fig, ax = plt.subplots()
ax.imshow(correlations[i_plane])
ax.axis("off")
ax.plot(coords[:,1], coords[:,0], "r.")
fig.savefig("correlation_local_maxima.png")

## Extract traces around maxima

You can loop around the maxima coordinates and take a small rectangular window around it. For better results use a gaussian kernel

In [ ]:
traces = np.empty((coords.shape[0], ar_imaging.shape[0]))

# fill the traces array defined above


In [ ]:
time = np.arange(traces.shape[1])/fps

In [ ]:
fig, ax = plt.subplots()
img = ax.imshow(zscore(traces, 1), aspect="auto", vmin=-3, vmax=3, cmap="RdBu_r", extent=(0, time[-1], traces.shape[0], 0))
ax.set_xlabel("time [s]")
ax.set_ylabel("trace number")
fig.colorbar(img)

In [ ]:
fig.savefig("traces_anat_1.png")

The result should look like this:

![](traces_anat_1.png)

Use ipywidgets interact to browse through the individual traces

In [ ]:
from ipywidgets import interact

In [ ]:
fig, ax = plt.subplots()
@interact
def browse_trace(i_trace:(0, traces.shape[0]-1)):
    ax.clear()
    ax.plot(time, traces[i_trace])

* Extend this to extract traces all at once across planes

## Correlation growing

Iterate through the local maxima (you can just find the global maximum, and remove it from consideration) and add neighbouring voxels if they correlate well with the first one chosen.

In [ ]:
n_max_rois = 100
for i in range(n_max_rois):
    # iteratively add ROIs
    pass

# Non-negative matrix factorisation

select a subset of data (NMF is expensive)

In [ ]:
y_start = 120
x_start = 43
width = 50
t_max = 960
ar_subset = ar_imaging[:960, 1:, y_start:y_start+width, x_start:x_start+width]

In [ ]:
display_array(ar_subset, vmin=0, vmax=200);

Reshape the data so that it can be factorised

In [ ]:
reshaped = ar_subset.reshape(ar_subset.shape[0], -1)

In [ ]:
from sklearn.decomposition import NMF

We choose 6 componenets

In [ ]:
nmf_dec = NMF(6, init="nndsvd")

In [ ]:
W = nmf_dec.fit_transform(reshaped)

In [ ]:
H = nmf_dec.components_

In [ ]:
spatial = H.reshape(H.shape[0], *ar_subset.shape[1:])

In [ ]:
display_array(spatial)

In [ ]:
plt.figure()
plt.plot(zscore(W, 0) + np.arange(W.shape[1])[None, :]*2);

### Compare with the traces from the previous method

Which ROIs are in both (search through the coordinates of local maxima)

In [ ]:
sel_within = np.logical_and(np.logical_and(coords[:,0] > y_start, coords[:,0] < y_start + width),
                            np.logical_and(coords[:,1] > x_start, coords[:,1] < x_start + width))
sel_coords = coords[sel_within, :]
fig, ax = plt.subplots()
ax.imshow(correlations[i_plane])
ax.axis("off")
ax.plot([x_start, x_start+width, x_start+width, x_start, x_start],[y_start, y_start, y_start+width, y_start+width, y_start], "w")
ax.plot(sel_coords[:,1], sel_coords[:,0], ".r")

In [ ]:
traces_nonfactorised = traces[sel_within, :t_max].T

In [ ]:
plt.figure()
plt.plot(zscore(traces_nonfactorised, 0) + np.arange(traces_nonfactorised.shape[1])[None, :]*2);

### Reconstruct the imaging

It's only a matrix multiplication + reshaping

In [ ]:
ar_reconstructed = # fill this in

In [ ]:
display_array(np.concatenate([ar_subset, ar_reconstructed], 3), vmin=0, vmax=150)

## Extra credit

* Investigate how different initialization mentohds impact the quality of the results
* Devise an intialisation step from the previous exercise (the matrices)
* Investigate the performance properties of the NMF coordinate descent algorithm (the default in scikit-learn)
* Suggest a way to merge results from two adjacent processing blocks